In [1]:
### Generate equations of motion for 3D double pendulum ###
import sys
sys.path.insert(0, "..")
import sympy.physics.mechanics as me
import sympy as sp
import gen_matlab_class as gm

t = sp.symbols('t')
q = me.dynamicsymbols('q0A, q1A, q2A, q3A, q0B, q1B, q2B, q3B')  # quaternions
u = me.dynamicsymbols('u1A, u2A, u3A, u1B, u2B, u3B')  # angular velocities
l, m, g, Ixx, Iyy, Izz, c = sp.symbols('l m g Ixx Iyy Izz c')  # constants

# set ground frame
N = me.ReferenceFrame('frame_ground')
N0 = me.Point('point_ground')
N0.set_vel(N,0)

# create bodies reference frames
A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')
mA = me.Point('mA') #center of mass of body A
mB = me.Point('mB') #center of mass of body B
ABj = me.Point('ABj') #joint position

# rotate joint by quaternion and apply angular velocities
# first joint is in the world frame center
A.orient(N, 'Quaternion', [q[0], q[1], q[2], q[3]])
A.set_ang_vel(N, u[0]*A.x + u[1]*A.y + u[2]*A.z)

# set masscenter of the first body
mA.set_pos(N0, -l/2 * A.z)
mA.v2pt_theory(N0,N,A)

# set the position of the joint between the two bodies
ABj.set_pos(N0, -l * A.z)
ABj.v2pt_theory(N0,N,A)

# orient second body from the first body
B.orient(A, 'Quaternion', [q[4], q[5], q[6], q[7]])
B.set_ang_vel(A, u[3]*B.x + u[4]*B.y + u[5]*B.z)
#set the masscenter of the second body
mB.set_pos(ABj, -l/2 * B.z)
mB.v2pt_theory(ABj,N,B)


I1 = me.inertia(A, Ixx, Iyy, Izz)
I2 = me.inertia(B, Ixx, Iyy, Izz)

BODY = []
BODY.append(me.RigidBody('Abody', mA, A, m, (I1, mA)))
BODY.append(me.RigidBody('Bbody', mB, B, m, (I2, mB)))

#define gravity force
FG1 = [(mA, -m * g * N.z)]
FG2 = [(mB, -m * g * N.z)]

#define damping in joints
DAMP = []
DAMP.append((A,-c*(u[0]*A.x + u[1]*A.y + u[2]*A.z)))
DAMP.append((B,-c*(u[3]*B.x + u[4]*B.y + u[5]*B.z)))
DAMP.append((A, c*(u[3]*B.x + u[4]*B.y + u[5]*B.z)))

# set kinematic differential equations - see Quaternions and Dynamics, page 9, equation 18
kindeq = []
for i in range(2):
    kindeq.append(q[0+i*4].diff(t) - 0.5 * (-u[0+i*3]*q[1+i*4] - u[1+i*3]*q[2+i*4] - u[2+i*3]*q[3+i*4]))
    kindeq.append(q[1+i*4].diff(t) - 0.5 * ( u[0+i*3]*q[0+i*4] + u[2+i*3]*q[2+i*4] - u[1+i*3]*q[3+i*4]))
    kindeq.append(q[2+i*4].diff(t) - 0.5 * ( u[1+i*3]*q[0+i*4] - u[2+i*3]*q[1+i*4] + u[0+i*3]*q[3+i*4]))
    kindeq.append(q[3+i*4].diff(t) - 0.5 * ( u[2+i*3]*q[0+i*4] + u[1+i*3]*q[1+i*4] - u[0+i*3]*q[2+i*4]))
F = FG1+FG2
kindeq
KM = me.KanesMethod(N, q_ind=q, u_ind=u, kd_eqs=kindeq)
(fr, frstar) = KM.kanes_equations(BODY, (FG1+FG2+DAMP))

In [2]:
# substituting dynamicsymbols for symbolics so it can be printed into matlab
qm = sp.symbols('qm0:8')
um = sp.symbols('um1:7')
subs_q = {q[i]: qm[i] for i in range(len(q))}
subs_u = {u[i]: um[i] for i in range(len(u))}
mm = me.msubs(KM.mass_matrix_full,subs_q,subs_u)
fo = me.msubs(KM.forcing_full,subs_q,subs_u)

from importlib import reload
reload(gm)
#create a dictionary of constatns
parameters_dict = {l:'l',m:'m',g:'g',Ixx:'Ixx',Iyy:'Iyy',Izz:'Izz',c:'c'}
#generate matlab functions using function from gen_matlab_class.py
gm.MatlabFunction(function = fo,fun_name = 'fo_quat',assignto = 'fo_quat',coordinates = qm,speeds = um,inputs = {},parameters = parameters_dict)
gm.MatlabFunction(function = mm,fun_name = 'mm_quat',assignto = 'mm_quat',coordinates = qm,speeds = um,inputs = {},parameters = parameters_dict)

In [3]:
# Notes
# vector of coordinates
print('q vector:',KM.q)
# vector of angular velocities
print('u vector:',KM.u)
# ODE is in the form: M*x_dot = fe, where x_dot = [q,u]_dot

q vector: Matrix([[q0A(t)], [q1A(t)], [q2A(t)], [q3A(t)], [q0B(t)], [q1B(t)], [q2B(t)], [q3B(t)]])
u vector: Matrix([[u1A(t)], [u2A(t)], [u3A(t)], [u1B(t)], [u2B(t)], [u3B(t)]])


In [15]:
## Euler
# just different way how create equations of motion,
# here one more speed is introduced, but this approach has a singularity when q0=0

import sympy.physics.mechanics as me
import sympy as sp
import gen_matlab_class as gm

t = sp.symbols('t')
q = me.dynamicsymbols('q1A, q2A, q3A, q1B, q2B, q3B')  # quaternion
u = me.dynamicsymbols('u1A, u2A, u3A, u1B, u2B, u3B')  # angular velocities
u0A, u0B = me.dynamicsymbols('u0A u0B')
l, m, g, Ixx, Iyy, Izz, c = sp.symbols('l m g Ixx Iyy Izz c')  # 

N = me.ReferenceFrame('frame_ground')
N0 = me.Point('point_ground')
N0.set_vel(N,0)

A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')
mA = me.Point('mA')
mB = me.Point('mB')
ABj = me.Point('ABj')


A.orient_body_fixed(N, [q[0], q[1], q[2]], 'YZY')
ang_vel1 = (A.ang_vel_in(N))
 
kinematical1 = sp.Matrix([
    q[0].diff()-u[0],
    q[1].diff()-u[1],
    q[2].diff()-u[2],
])

B.orient_body_fixed(A, [q[3], q[4], q[5]], 'YZX')
ang_vel2 = (B.ang_vel_in(A))

# A_w_B = B.ang_vel_in(A)

kinematical2 = sp.Matrix([
    q[3].diff()-u[3],
    q[4].diff()-u[4],
    q[5].diff()-u[5],
])


mA.set_pos(N0, -l/2 * A.z)
mA.v2pt_theory(N0,N,A)
FG1 = [(mA, -m * g * N.z)]

ABj.set_pos(N0, -l * A.z)
ABj.v2pt_theory(N0,N,A)


mB.set_pos(ABj, -l/2 * B.z)
mB.v2pt_theory(ABj,N,B)


I1 = me.inertia(A, Ixx, Iyy, Izz)
I2 = me.inertia(B, Ixx, Iyy, Izz)

BODY = []
BODY.append(me.RigidBody('Abody', mA, A, m, (I1, mA)))
BODY.append(me.RigidBody('Bbody', mB, B, m, (I2, mB)))

kinematical = sp.Matrix([[kinematical1],[kinematical2]])

FG2 = [(mB, -m * g * N.z)]
DAMP = []
DAMP.append((A,-c*(ang_vel1.dot(A.x)*A.x + ang_vel1.dot(A.y)*A.y + ang_vel1.dot(A.z)*A.z)))
DAMP.append((B,-c*(ang_vel2.dot(B.x)*B.x + ang_vel2.dot(B.y)*B.y + ang_vel2.dot(B.z)*B.z)))
DAMP.append((A, c*(ang_vel2.dot(B.x)*B.x + ang_vel2.dot(B.y)*B.y + ang_vel2.dot(B.z)*B.z)))


F = FG1+FG2
kane = me.KanesMethod(
    N,
    q,
    u,
    kd_eqs=kinematical,
)
(fr, frstar) = kane.kanes_equations(BODY, (FG1+FG2+DAMP))

In [16]:
# substituting dynamicsymbols for symbolics so it can be printed into matlab
qm = sp.symbols('qm1:7')
um = sp.symbols('um1:7')
subs_q = {q[i]: qm[i] for i in range(len(q))}
subs_u = {u[i]: um[i] for i in range(len(u))}
mm = me.msubs(kane.mass_matrix_full,subs_q,subs_u)
fo = me.msubs(kane.forcing_full,subs_q,subs_u)

from importlib import reload
reload(gm)
parameters_dict = {l:'l',m:'m',g:'g',Ixx:'Ixx',Iyy:'Iyy',Izz:'Izz',c:'c'}
gm.MatlabFunction(function = fo,fun_name = 'fo_eul',assignto = 'fo_eul',coordinates = qm,speeds = um,inputs = {},parameters = parameters_dict)
gm.MatlabFunction(function = mm,fun_name = 'mm_eul',assignto = 'mm_eul',coordinates = qm,speeds = um,inputs = {},parameters = parameters_dict)

In [19]:
ang_vel2.dot(B.x)

sin(q2B(t))*Derivative(q1B(t), t) + Derivative(q3B(t), t)

In [32]:
ang_vel2.dot(B.z)

-sin(q3B(t))*cos(q2B(t))*Derivative(q1B(t), t) + cos(q3B(t))*Derivative(q2B(t), t)

In [33]:
ang_vel1.dot(A.z)

sin(q2A(t))*sin(q3A(t))*Derivative(q1A(t), t) + cos(q3A(t))*Derivative(q2A(t), t)